# Ваш первый классификатор

Напоминание: $X$ - набор объектов, с признаками
$Y$ - набор правильных ответов для каждого объекта

Цель - построить алгоритм $a(X)$ предсказать для каждого обхекта $X$ правильный ответ с

$$y = a([x_1, x_2, x_3, ...])$$


Простейшая задача классификации - бинарная классификация, где ответом для каждого объекта будет 0 или 1, "True" или "False"; "Да" или "Нет", "Вернет кредит" или "Не вернет кредит"; и тд

В этом упражнении мы обучим бинарный классификатор, который будет предсказывать, следует ли пациенту пройти обследование на диабет, основываясь на некоторых медицинских данных.


### Загрузка и исследование данных

Загрузите из репозитория файл diabetes.csv и загрузите его сюда, нажав слева на ячейку с папкой
Выполните следующую ячейку, чтобы загрузить CSV-файл с данными о пациентах в датафрейм **Pandas**:

In [ ]:
import pandas as pd
import os
diabetes = pd.read_csv('diabetes.csv')
diabetes.head()

Эти данные состоят из диагностической информации о некоторых пациентах, которые прошли тест на диабет. При необходимости прокрутите страницу вправо и обратите внимание, что последний столбец набора данных (**Diabetic**) содержит значение ***0*** для пациентов с отрицательным тестом на диабет и ***1*** для пациентов с положительным тестом. Это та метка, которую мы будем обучать нашу модель предсказывать; большинство других столбцов

 (**Беременность**, **Уровень глюкозы**, **Давление** и так далее) - это признаки, которые мы будем использовать для предсказания метки **Diabetic**.

Давайте отделим признаки от меток - назовем признаки ***X***, а метку ***y***:

In [ ]:
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
label = 'Diabetic'
X, y = diabetes[features].values, diabetes[label].values

for n in range(0,4):
    print("Patient", str(n+1), "\n  Features:",list(X[n]), "\n  Label:", y[n])

Теперь давайте сравним распределения признаков для каждого значения метки.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
for col in features:
    diabetes.boxplot(column=col, by='Diabetic', figsize=(6,6))
    plt.title(col)
plt.show()

Для некоторых характеристик наблюдается заметная разница в распределении для каждого значения метки. В частности, для признаков «Беременность» и «Возраст» распределения для пациентов с диабетом заметно отличаются от распределений для пациентов без диабета. Эти признаки могут помочь предсказать, является ли пациент диабетиком или нет.

### Разделим наши данные

Наш набор данных содержит известные значения метки, поэтому мы можем использовать его для обучения классификатора, чтобы он находил статистическую связь между признаками и значением метки; но как мы узнаем, хороша ли наша модель? Как мы узнаем, что она будет предсказывать правильно, если мы используем ее с новыми данными, на которых она не обучалась? Мы можем воспользоваться тем, что у нас есть большой набор данных с известными значениями меток, использовать только часть из них для обучения модели и отложить часть для тестирования обученной модели - это позволит нам сравнить предсказанные метки с уже известными метками в тестовом наборе.

В Python пакет **scikit-learn** содержит большое количество функций, которые мы можем использовать для построения модели машинного обучения, включая функцию **train_test_split**, которая обеспечивает случайное разделение данных для обучения и тестирования. С ее помощью мы разделим данные на 70 % для обучения и 30 % для тестирования.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.30,
                                                    random_state=0)

print ('Training cases: %d\nTest cases: %d' % (X_train.shape[0], X_test.shape[0]))

### Обучим нашу первую модель!


Итак, теперь мы готовы обучить нашу модель, подогнав обучающие признаки (**X_train**) к обучающим меткам (**y_train**). Существуют различные алгоритмы, которые мы можем использовать для обучения модели. В этом примере мы будем использовать *Логистическую регрессию*, которая является хорошо зарекомендовавшим себя алгоритмом классификации.

> **Примечание**: Параметры алгоритмов машинного обучения обычно называются *гиперпараметрами* (для специалиста по изучению данных *параметры* - это значения в самих данных, а *гиперпараметры* определяются извне, на основе данных).

In [ ]:
from sklearn.linear_model import LogisticRegression

reg = 0.01
# C=1/reg, solver="liblinear" - гиперпараметры здесь
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)
print(model)

Теперь, когда мы обучили модель на обучающих данных, мы можем использовать тестовые данные, которые мы отложили, чтобы оценить, насколько хорошо она предсказывает. В этом нам снова поможет **scikit-learn**. Давайте начнем с использования модели для предсказания меток для нашего тестового набора и сравним предсказанные метки с известными метками:

In [ ]:
predictions = model.predict(X_test)
print('Предсказания: ', predictions)
print('Истинные ответы:    ' ,y_test)

Массивы меток слишком длинные, чтобы выводить их в блокноте, поэтому мы можем сравнить только несколько значений. Даже если бы мы распечатали все предсказанные и фактические метки, их было бы слишком много, чтобы это было разумным способом оценки модели. К счастью, у **scikit-learn** есть для этого функции, и библиотека предоставляет некоторые метрики, которые мы можем использовать для оценки модели.

Самое очевидное, что вы можете захотеть сделать, это проверить *accuracy* предсказаний - простыми словами, какую долю меток модель предсказала правильно?

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

Точность возвращается в виде десятичного значения - значение 1.0 означает, что модель сделала 100 % правильных предсказаний; в то время как точность 0.0, ну, довольно бесполезна!

Точность кажется разумной метрикой для оценки (и в некоторой степени так оно и есть), но нужно быть осторожным и не делать слишком много выводов из точности классификатора. Помните, что это просто показатель того, сколько случаев было предсказано правильно. Предположим, только 3 % населения страдают диабетом. Можно создать классификатор, который всегда предсказывает только 0, и он будет точным на 97 %, но не слишком полезным для выявления пациентов с диабетом!

К счастью, есть и другие метрики, которые позволяют узнать немного больше о том, как работает наша модель. В Scikit-Learn есть возможность создать *отчет о классификации*, который дает больше информации, чем только сырая точность.

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, predictions))

Отчет о классификации включает следующие метрики для каждого класса (0 и 1)

> обратите внимание, что строка заголовка может не совпадать со значениями!

* * *Precision*: Какая часть предсказаний, сделанных моделью для этого класса, оказалась верной?
* *Recall*: Из всех экземпляров этого класса в тестовом наборе данных, какую часть идентифицировала модель?
* *F1-Score*: Средняя метрика, учитывающая precision и recall.
* *Support*: Сколько экземпляров этого класса имеется в тестовом наборе данных?

Отчет о классификации также включает средние показатели для этих метрик, в том числе средневзвешенное значение, которое позволяет учесть дисбаланс в количестве случаев каждого класса.

Поскольку это *бинарная* задача классификации, класс ***1*** считается *позитивным*, и его точность и отзыв представляют особый интерес - они фактически отвечают на вопросы:

- Из всех пациентов, которым модель предсказала диабет, сколько на самом деле являются диабетиками?
- Сколько из всех пациентов, которые на самом деле являются диабетиками, было выявлено моделью?

Вы можете получить эти значения самостоятельно, используя метрики **precision_score** и **recall_score** в scikit-learn (которые по умолчанию предполагают бинарную модель классификации).

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Общий Precision:",precision_score(y_test, predictions))
print("Общий Recall:",recall_score(y_test, predictions))

Показатели precision и recall определяются на основе четырех возможных результатов прогнозирования:
* * Истинные положительные результаты (True Positive): Предсказанная метка и фактическая метка равны 1.
* * Ложные положительные результаты* (False Positive): Предсказанная метка равна 1, но фактическая метка равна 0.
* *Ложные отрицательные* (False Negative): Предсказанная метка равна 0, а фактическая - 1.
* * Истинные отрицательные значения* (True Negative): Предсказанная метка и фактическая метка равны 0.

Эти метрики обычно собираются в таблицу для тестового набора данных и отображаются вместе в виде *матрицы ошибок*, которая имеет следующий вид:

<table style="border: 1px solid black;">
    <tr style="border: 1px solid black;">
        <td style="border: 1px solid black;color: black;" bgcolor="lightgray">TN</td><td style="border: 1px solid black;color: black;" bgcolor="white">FP</td>
    </tr>
    <tr style="border: 1px solid black;">
        <td style="border: 1px solid black;color: black;" bgcolor="white">FN</td><td style="border: 1px solid black;color: black;" bgcolor="lightgray">TP</td>
    </tr>
</table>

Обратите внимание, что правильные (*истинные*) предсказания образуют диагональную линию сверху слева направо - эти цифры должны быть значительно выше, чем *ложные* предсказания, если модель хоть сколько-нибудь хороша.

В Python вы можете использовать функцию **sklearn.metrics.confusion_matrix**, чтобы найти эти значения для обученного классификатора:

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)
print (cm)

До сих пор мы рассматривали предсказания модели как метки классов 1 или 0. На самом деле все немного сложнее. Статистические алгоритмы машинного обучения, такие как логистическая регрессия, основаны на *вероятности*; поэтому то, что на самом деле предсказывает бинарный классификатор, - это вероятность того, что метка является истинной (**P(y)**), и вероятность того, что метка является ложной (1 - **P(y)**). Пороговое значение 0,5 используется для определения того, является ли предсказанная метка 1 (если *P(y) > 0,5*) или 0 (если *P(y) <= 0,5*). Вы можете использовать метод **predict_proba**, чтобы увидеть пары вероятностей для каждого случая:

In [ ]:
y_scores = model.predict_proba(X_test)
print(y_scores)

Решение о том, считать ли предсказание 1 или 0, зависит от порогового значения, с которым сравниваются предсказанные вероятности. Если мы изменим порог, это повлияет на предсказания и все 4 значения (TP, TN, FP, FN), а значит, изменит метрики в матрице ошибок. Обычный способ оценки классификатора заключается в изучении *коэффициента истинных положительных результатов* (True Positive Rate) и *коэффициента ложных положительных результатов* (False Positive Rate) для ряда возможных порогов.

Формулы, по которым считаются эти две величины:


$$ FPR = \frac{FP}{FP + TN} $$

$$ TPR = \frac{TP}{FN + TP} $$

ничего формула TPR не напоминает?

Затем эти показатели строятся по всем возможным пороговым значениям, чтобы сформировать график, известный как ROC, как показано ниже:

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

fig = plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

График ROC показывает кривую истинных и ложных положительных результатов для различных пороговых значений от 0 до 1. Идеальный классификатор будет иметь кривую, которая идет прямо вверх по левой стороне и прямо через верх. Диагональная линия, пересекающая график, представляет собой вероятность правильного предсказания при случайном прогнозе 50/50; поэтому очевидно, что кривая должна быть выше этого значения (иначе ваша модель не лучше простого угадывания!).

Площадь под кривой (AUC - Area Under Curve) - это значение от 0 до 1, которое оценивает общую эффективность модели. Чем ближе к 1 это значение, тем лучше модель. И снова, scikit-learn включает функцию для расчета этой метрики.

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

### Выполните предварительную обработку в конвейере

В данном случае ROC-кривая и ее AUC показывают, что модель работает лучше, чем случайное предположение, что неплохо, учитывая, что мы выполнили очень мало предварительной обработки данных.

На практике обычно выполняется некоторая предварительная обработка данных, чтобы алгоритму было проще подобрать к ним модель. Существует огромное количество преобразований, которые можно выполнить, чтобы подготовить данные к моделированию, но мы ограничимся несколькими распространенными методами:

- Нормализация (масштабирование) числовых признаков, чтобы они находились в одном масштабе. Это не позволит признакам с большими значениями заставлять модель подбирать коэффициенты, непропорционально влияющие на прогнозы.


- Кодирование категориальных переменных. Например, используя технику *one-hot-encoding*, можно создавать отдельные бинарные признаки (истина/ложь) для каждого возможного значения категории.

Для применения этих предварительных преобразований мы воспользуемся функцией Scikit-Learn под названием *pipelines*. Она позволяют определить набор шагов предварительной обработки, которые завершаются алгоритмом. Затем вы можете запустить весь этого процесс на данным, так что модель будет включать в себя все шаги предварительной обработки, а также непосредственно алгоритм классификации. Это полезно, поскольку, когда мы захотим использовать модель для предсказания значений по новым данным, нам нужно будет применить те же преобразования, что были применены к обучающим данным (понимаете, почему?)

In [ ]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np

# запускаем нормализацию численных признаков - для этого указали их индексы
numeric_features = [0,1,2,3,4,5,6]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# определеим, что делаем с категориальным признаком - возраст
categorical_features = [7]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# объединяем два предыдущих шага
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Создаем пайплайн предобработки и обучения
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', LogisticRegression(C=1/reg, solver="liblinear"))])


# Запускаем его на обучающих данных
model = pipeline.fit(X_train, (y_train))
print (model)

In [ ]:
# Получаем предсказания для тестовой выборки
predictions = model.predict(X_test)
y_scores = model.predict_proba(X_test)

# Подсчитываем метрики чтобы понять, как хорошо справилась модель
cm = confusion_matrix(y_test, predictions)
print ('Матрица ошибок:\n',cm, '\n')
print('Accuracy:', accuracy_score(y_test, predictions))
print("Общий Precision:",precision_score(y_test, predictions))
print("Общий Recall:",recall_score(y_test, predictions))
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

# ROC-кривая
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

fig = plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()



Результаты выглядят немного лучше, так что очевидно, что предварительная обработка данных имеет значение.

### Попробуйте другой алгоритм

Теперь давайте попробуем другой алгоритм. Ранее мы использовали алгоритм логистической регрессии, который является *линейным* алгоритмом. Существует множество видов алгоритмов классификации, которые мы могли бы попробовать, включая:


- **Support Vector Machine algorithms**
- **Tree-based algorithms**
- **Ensemble algorithms**


In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('logregressor', RandomForestClassifier(n_estimators=100))])

model = pipeline.fit(X_train, (y_train))
print (model)

In [ ]:
predictions = model.predict(X_test)
y_scores = model.predict_proba(X_test)
cm = confusion_matrix(y_test, predictions)
print ('Матрица ошибок:\n',cm, '\n')
print('Accuracy:', accuracy_score(y_test, predictions))
print("Общий Precision:",precision_score(y_test, predictions))
print("Общий Recall:",recall_score(y_test, predictions))
auc = roc_auc_score(y_test,y_scores[:,1])
print('\nAUC: ' + str(auc))

fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

fig = plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()


### Использование моделей для предсказаний

In [ ]:
import joblib
import os

# Сохраняем модель для дальнейшего использования
os.mkdir('models')
filename = './models/diabetes_model.pkl'
joblib.dump(model, filename)

When we have some new observations for which the label is unknown, we can load the model and use it to predict values for the unknown label:

In [ ]:
# Загружаем модель
model = joblib.load(filename)

# создадим данные нового пациента
X_new = np.array([[2,180,74,24,21,23.9091702,1.488172308,22]])
print ('Пациент: {}'.format(list(X_new[0])))

pred = model.predict(X_new)

print('Предсказанный ответ - {}'.format(pred[0]))